In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

In [2]:
df = pd.read_csv("OASIS4_data_clinical.csv")

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt

# Your plotting code here

plt.show()  # Display the plot in the Jupyter Notebook output


In [4]:
# # Import the necessary module
# from ydata_profiling import ProfileReport

# # Create a ProfileReport object
# report = ProfileReport(df)

# # Generate the data profiling report
# report.to_file('oasis_4_2.html')

In [5]:
df.dropna(inplace=True)

In [6]:
df.drop('oasis_id' , axis=1,inplace=True)
df.drop('demographics_id', axis=1,inplace=True)
df.drop('demographics_firstvisit', axis=1,inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 0 to 662
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sex                   660 non-null    int64  
 1   age                   660 non-null    int64  
 2   edu                   660 non-null    int64  
 3   race                  660 non-null    int64  
 4   hispanic              660 non-null    int64  
 5   marriage              660 non-null    int64  
 6   declong               660 non-null    float64
 7   decage                660 non-null    int64  
 8   smoke                 660 non-null    int64  
 9   height                660 non-null    float64
 10  weight                660 non-null    float64
 11  bmi                   660 non-null    float64
 12  health_history1       660 non-null    int64  
 13  health_history2       660 non-null    int64  
 14  health_history3       660 non-null    int64  
 15  health_history4       6

In [8]:
# check all the columns in df and convert all the int columns to float, except 'cdr'
for col in df.columns:
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64')

##### Train-Test Split

In [9]:
df.drop('hispanic', axis=1,inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 0 to 662
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sex                   660 non-null    float64
 1   age                   660 non-null    float64
 2   edu                   660 non-null    float64
 3   race                  660 non-null    float64
 4   marriage              660 non-null    float64
 5   declong               660 non-null    float64
 6   decage                660 non-null    float64
 7   smoke                 660 non-null    float64
 8   height                660 non-null    float64
 9   weight                660 non-null    float64
 10  bmi                   660 non-null    float64
 11  health_history1       660 non-null    float64
 12  health_history2       660 non-null    float64
 13  health_history3       660 non-null    float64
 14  health_history4       660 non-null    float64
 15  health_history5       6

In [11]:
df.drop('race', axis=1,inplace=True)

In [12]:
# show unique values in final_dx
df['final_dx'].value_counts()

Alzheimer Disease Dementia                  219
Uncertain - AD possible                     118
MCI                                          49
Cognitively Normal                           47
AD/Vascular                                  40
Mood/polypharmacy/sleep                      31
Early Onset AD                               27
FTD                                          25
Non-Neurodegenerative Neurologic Disease     22
Vascular Cognitive Impairment (VCI)          16
Other - Miscellaneous                        15
AD Variant                                   12
DLB                                          11
Other Non-AD Neurodegenerative Disorder      11
AD+Non Neurodegenerative                     10
PPA                                           7
Name: final_dx, dtype: int64

In [13]:
# encode final_dx
df.drop('final_dx', axis=1,inplace=True)

In [14]:
from sklearn.model_selection import train_test_split
x = df.drop(['final_dx_categorized'], axis=1)
y = df[['final_dx_categorized']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [15]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = sm.fit_resample(x_train, y_train)

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_resampled = scaler.fit_transform(x_train_resampled)
x_test = scaler.transform(x_test)

In [17]:
# import csv
# import os
# class CSVLogger:
#     def __init__(self, filename, fieldnames):
#         self.filename = filename
#         self.fieldnames = fieldnames

#         # Create and initialize the CSV file
#         if not os.path.isfile(self.filename):
#             with open(self.filename, 'w', newline='') as csvfile:
#                 writer = csv.DictWriter(csvfile, fieldnames=self.fieldnames)
#                 writer.writeheader()
#         else:
#             with open(self.filename, 'a', newline='') as csvfile:
#                 writer = csv.DictWriter(csvfile, fieldnames=self.fieldnames)
#                 writer.writerow({})

#     def log(self, data):
#         # Append data to the CSV file
#         with open(self.filename, 'a', newline='') as csvfile:
#             writer = csv.DictWriter(csvfile, fieldnames=self.fieldnames)
#             writer.writerow(data)

In [18]:
# # create a CSVLogger object
# csv_logger = CSVLogger( '.csv', ['model', 'accuracy'] )


### Models

##### Logistic Regression

In [19]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

# Train the model
model = LogisticRegression()
model.fit(x_train_resampled, y_train_resampled)

# Make predictions and evaluate
predictions = model.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")
conf_matrix = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(conf_matrix)

# log = {'model': 'Logistic Regression with SMOTE', 'accuracy': accuracy}
# csv_logger.log(log)

Accuracy: 0.13
Confusion Matrix:
[[0 0 0 0 1 0 0 0 0 2 1 0 1 3 1]
 [0 8 4 0 0 0 1 8 1 5 4 0 1 9 5]
 [0 0 1 0 1 1 1 1 0 0 0 0 0 1 0]
 [0 0 2 2 0 0 0 0 1 2 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 2 1 0]
 [0 1 0 1 0 1 1 0 0 1 1 2 0 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 3 2 1 0 0 0 2 0 0 1 0 0 1 1]
 [1 2 1 2 0 1 2 2 2 3 0 1 2 0 3]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 1 0 0 1 1 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 1 0]
 [0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]]


C:\Users\Ananya\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ananya\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##### SVM

In [20]:
# # svm
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score, confusion_matrix
# from sklearn.model_selection import train_test_split

# # Perform SMOTE with reduced k_neighbors
# smote = SMOTE(sampling_strategy='auto', k_neighbors=1)  # Adjust k_neighbors as needed
# x_resampled, y_resampled = smote.fit_resample(x_train, y_train)

# # Train the model
# model = SVC(kernel='linear', random_state=42)
# model.fit(x_resampled, y_resampled)

# # Make predictions and evaluate
# predictions = model.predict(x_test)
# accuracy = accuracy_score(y_test, predictions)
# print(f"Accuracy: {accuracy:.2f}")
# conf_matrix = confusion_matrix(y_test, predictions)
# print("Confusion Matrix:")
# print(conf_matrix)

# # log = {'model': 'SVM with SMOTE', 'accuracy': accuracy}
# # csv_logger.log(log)

##### RF

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create and train the Random Forest model
model = RandomForestClassifier()
model.fit(x_train, y_train)

# Make predictions
y_pred = model.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# log = {'model': 'Random Forest', 'accuracy': accuracy}
# csv_logger.log(log)

C:\Users\Ananya\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Accuracy: 0.05


C:\Users\Ananya\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


##### XG Boost

In [22]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Create an XGBoost regressor
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)

# Fit the model on the training data
model.fit(x_train, y_train)

# Make predictions on the test data
y_pred = model.predict(x_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(mse)
# log = {'model': 'XGBoost', 'accuracy': mse}
# csv_logger.log(log)


34.448431425088884


In [23]:
from sklearn.ensemble import GradientBoostingClassifier
# split into x and y where y is 'Group' 

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=0)
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")
# log = {'model': 'Gradient Boost', 'accuracy': accuracy}
# csv_logger.log(log)

C:\Users\Ananya\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.02


C:\Users\Ananya\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


### Other Models

##### KNN

In [24]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()

model.fit(x_train, y_train)

predictions = model.predict(x_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.07


C:\Users\Ananya\AppData\Roaming\Python\Python311\site-packages\sklearn\neighbors\_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Ananya\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [25]:
# shape of x and y
print(x.shape)
print(y.shape)

(660, 20)
(660, 1)


##### NN

In [27]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(20,)),
    tf.keras.layers.Dense(16, activation='softmax')
])


In [28]:
# Assuming you have a test dataset for prediction (x_test)
# Make sure that x_test has the same number of features as your training data (20 in this case)

# Use the trained model to make predictions
predictions = model.predict(x_test)

# 'predictions' will contain the model's output, which are the class probabilities
# You can choose the class with the highest probability as the predicted class
predicted_classes = predictions.argmax(axis=1)

# 'predicted_classes' will contain the predicted class labels

# If you have a mapping of class labels, you can map the labels back to their original values
# For example, if 'final_dx_categorized' had labels like 0, 1, 2, etc.
# and you have a mapping like {0: 'ClassA', 1: 'ClassB', 2: 'ClassC', ...}
# you can do something like this to get the class labels:
# predicted_class_labels = [your_mapping[class_idx] for class_idx in predicted_classes]


5/5 [==============================] - 0s 3ms/step


In [29]:
# accuracy
accuracy = accuracy_score(y_test, predicted_classes)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.06
